In [ ]:
%pip install tensorflow==2.13.0
%pip install tensorflow-model-optimization==0.7.5

# 1. Performance testing

In [ ]:
import tensorflow as tf 
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build and train a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

print("Training model...")
model.fit(x_train, y_train, epochs=3, validation_split=0.2, verbose=1)

# Get predictions on test set
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)  # Convert probabilities to class labels
y_true = y_test  # Actual labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
# For multi-class, use 'macro' or 'weighted' instead of 'binary'
precision = precision_score(y_true, y_pred, average='weighted')

print(f'\nAccuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
import time

# Create sample input data - use a batch of test images
input_data = x_test[:100]  # Take first 100 test images

# Measure response time
start_time = time.time()
predictions = model.predict(input_data)
end_time = time.time()

response_time = end_time - start_time
avg_response_time = response_time / len(input_data)

print(f'Total Response Time: {response_time:.4f} seconds')
print(f'Average Response Time per image: {avg_response_time:.6f} seconds')
print(f'Predictions per second: {len(input_data) / response_time:.2f}')

# 2. Prunning

In [ ]:
import tensorflow_model_optimization as tfmot

# Define pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0, final_sparsity=0.5, begin_step=0, end_step=1000
    )
}

# Apply pruning to the Sequential model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Retrain the pruned model to finalize pruning
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
pruned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test), callbacks=callbacks)

# Strip pruning wrappers to remove pruning-specific layers and metadata
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# 3 Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()

# 4. Feature selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Flatten the MNIST images for sklearn (which expects 2D data)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Shape: (60000, 784)
x_test_flat = x_test.reshape(x_test.shape[0], -1)    # Shape: (10000, 784)

# Create and train logistic regression model
lr_model = LogisticRegression(max_iter=100, random_state=42)

# Use RFE to select top 100 features (pixels) - 784 pixels is too many
print("Applying Recursive Feature Elimination...")
rfe = RFE(lr_model, n_features_to_select=100, step=50)
rfe = rfe.fit(x_train_flat, y_train)

# Transform data to use only selected features
x_train_selected = rfe.transform(x_train_flat)
x_test_selected = rfe.transform(x_test_flat)

# Train the model with selected features
print("Training model with selected features...")
lr_model.fit(x_train_selected, y_train)

# Evaluate
accuracy = lr_model.score(x_test_selected, y_test)
print(f'\nAccuracy with {rfe.n_features_} selected features: {accuracy:.4f}')
print(f'Selected {rfe.n_features_} out of {x_train_flat.shape[1]} total features')

# 5. Stress testing

In [ ]:
for i in range(1000):
    print(f"{i} - Model prediction: {model.predict(input_data)}")